#### XBRL US API Example
This example obtains non-restated Revenue and Net Income for Apple, Microsoft, and Amazon for 2022 and 2023. 

##### Import Required Packages

In [1]:
import os, re, sys, json, requests, getpass, urllib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from datetime import datetime
from urllib.parse import urlencode

##### Obtain API Access - Please Enter Your Email, Client ID, and Client Secret

In [2]:
password = getpass.getpass(prompt = 'Enter Your XBRL US Password: ')

body_auth = {'username' : 'vac35@psu.edu', 
            'client_id': 'Obtain Client ID from XBRL Website', 
            'client_secret' : 'Obtain Client Secret from XBRL Website', 
            'password' : ''.join(password), 
            'grant_type' : 'password', 
            'platform' : 'ipynb' }

payload = urlencode(body_auth)
url = 'https://api.xbrl.us/oauth2/token'
headers = {"Content-Type": "application/x-www-form-urlencoded"}

res = requests.request("POST", url, data = payload, headers = headers)
auth_json = res.json()

if 'error' in auth_json:
    print ("Access Denied")
else:
    print ("Access Granted.")
    
access_token = auth_json['access_token']
refresh_token = auth_json['refresh_token']
newaccess = ''
newrefresh = ''

Access Granted.


##### Enter Parameters - Elements, Companies, Years, Filings, and Fields

In [3]:
XBRL_Elements = ['RevenueFromContractWithCustomerExcludingAssessedTax', 
     		     'NetIncomeLoss']

Companies = ['0000789019', ## Microsoft (MSFT)
           	 '0001018724', ## Amazon (AMZN)
             '0000320193'] ## Apple Inc. (AAPL)

Years = ['2022', '2023'] 

Filings = ['10-K']

Fields = ['entity.cik',
    	  'entity.name.sort(ASC)',
    	  'report.filing-date',
          'period.fiscal-year',
    	  'report.document-type',
          'concept.local-name',
          'fact.value',
          'unit']

##### Run The Query

In [4]:
Parameters = {'concept.local-name': ','.join(XBRL_Elements),
     	      'period.fiscal-period': 'Y',
     	      'period.fiscal-year': ','.join(Years),
    	      'unit': 'USD',
     	      'entity.cik': ','.join(Companies),
     	      'report.document-type': ','.join(Filings),}  

has_dimensions = 'FALSE'

if has_dimensions == 'ALL':
    dimension_options = ['TRUE', 'FALSE']
else:
    dimension_options = [has_dimensions]

search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'
    
all_res_list = []
for dimensions_param in dimension_options:

    print('Getting the data for: "fact.has-dimensions" = {}'.format(dimensions_param))

    done_retrieving_all_results = False
    offset = 0

    while not done_retrieving_all_results:

        Parameters['fact.has-dimensions'] = dimensions_param
        Parameters['fields'] = ','.join(Fields) + ',fact.offset({})'.format(offset) 

        res = requests.get(search_endpoint, params = Parameters, headers={'Authorization' : 'Bearer {}'.format(access_token)})
        
        res_json = res.json()
        res_list = res_json['data']
        all_res_list += res_list
        
        paging_dict = res_json['paging']

        print('Number of Observations Obtained: ', paging_dict['count'])

        if paging_dict['count'] >= 2000:
            offset += paging_dict['count']
        else:
            done_retrieving_all_results = True
    
df = pd.DataFrame(all_res_list)
print('Number of Observations: {}'.format(len(df)))


Getting the data for: "fact.has-dimensions" = FALSE
Number of Observations Obtained:  72
Number of Observations: 72


##### Keep Relevant Variables and Rename Them

In [5]:
df = df[['entity.cik', 'entity.name', 'period.fiscal-year', 'fact.value', 'concept.local-name', 'report.filing-date']]
df.columns = ['CIK', 'Company', 'Fiscal Year', 'value', 'account', 'filing']

##### Sort Data by CIK, Fiscal Year, Account, and Filing Date and Remove Duplicates

In [6]:
df = df.sort_values(by = ['CIK', 'Fiscal Year', 'filing', 'account'])
df = df.drop_duplicates(subset = ['CIK', 'Fiscal Year', 'filing', 'account'], keep = 'last')

##### Scale Values by Millions 

In [7]:
df['value'] = df['value'] / 1000000

##### Reorganize Data 

In [8]:
df = df.pivot_table(index=["CIK", "Company", "Fiscal Year"], columns="account", values="value", aggfunc='min')
df = df.rename_axis(None, axis=1)
df.reset_index(inplace=True)

##### Clean and Format Data

In [9]:
df.columns = ['CIK', 'Company', 'Fiscal Year', 'Net Income', 'Revenue']
df['Net Income'] = df['Net Income'].apply(lambda x: f'${x:,.2f}')
df['Revenue'] = df['Revenue'].apply(lambda x: f'${x:,.2f}')

In [10]:
df

,CIK,Company,Fiscal Year,Net Income,Revenue
0,0000320193,Apple Inc.,2022,"$99,803.00","$394,328.00"
1,0000320193,Apple Inc.,2023,"$96,995.00","$383,285.00"
2,0000789019,MICROSOFT CORPORATION,2022,"$72,738.00","$198,270.00"
3,0000789019,MICROSOFT CORPORATION,2023,"$72,361.00","$211,915.00"
4,0001018724,"AMAZON.COM, INC.",2022,"$-2,722.00","$513,983.00"
5,0001018724,"AMAZON.COM, INC.",2023,"$30,425.00","$574,785.00"


#### Refresh Token
The cell below is only needed to refresh an expired access token after 60 minutes. When the access token no longer returns results, run the cell below to refresh the access token or re-enter credentials by running the cell above. Until the refresh token process is needed, skip ahead to Make a Query.

In [ ]:
token = token if newrefresh != '' else refresh_token 

refresh_auth = {'client_id': 'a04fc50b-a62c-4e96-8578-6e71b3c9bc52', 
                'client_secret' : 'dc6805e2-f03b-4f68-808d-89cfffcfc469', 
                'grant_type' : 'refresh_token',
                'platform' : 'ipynb', 
                'refresh_token' : ''.join(token) }
refreshres = requests.post(url, data=refresh_auth)
refresh_json = refreshres.json()
access_token = refresh_json['access_token']
refresh_token = refresh_json['refresh_token']#print('access token: ' + access_token + 'refresh token: ' + refresh_token)
print('Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.')
print(access_token)